### Urdu Abusive content detection Revised - F1 - paper final
### Common Functions

In [2]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
#All the libraries that we need
import string
import nltk
from collections import Counter
import unicodedata
import re

from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import time

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB



#Common functions
punc=['…','،','۔','،','۔','؟' , '.', ',']
NFC = [('آ', 'آ'), ('ة', 'ۃ'), ('ـ', ''), ('ك', 'ک'), ('ه', 'ہ'), ('ۀ', 'ۂ'), 
       ('ى', 'ی'), ('ي', 'ی'), ('٠', '۰'), ('١', '۱'), ('٢', '۲'), 
       ('٣', '۳'), ('٤', '۴'), ('٥', '۵'), ('٦', '۶'), ('٧', '۷'), ('٨', '۸'), 
       ('٩', '۹'), ('ۓ', 'ئے'), ('ۓ', 'ئے'), ('ئ', 'ئی'),
       ("ِ", ""), ("ِ", ""),("ُ", ""),("َ", ""),("ْ", ""),("ٰ", ""),("ً", ""),("ّ", "")      
      ]

# load file into memory
def load_file(filename):
    file = open(filename, 'r', encoding='utf8')
    text = file.read()
    file.close()
    return text

def read_as_dict_for_lemma(file):
    text = load_file(file)
    lines = text.split("\n")
    wd = dict()
    for line in lines:
        toks = line.split(":")
        if len(toks)==0 or len(toks)==1:
            continue
        wd[toks[0].strip()] = toks[1].strip()
    return wd

#Loading Lemmatizer
dictionary = read_as_dict_for_lemma('Humayoun_lemmatizer.txt')

def lemma(word, dictionary):
    if word in dictionary:
        return dictionary[word]
    else:
        return word

def read_stopwords(file):
    text = load_file(file)
    tokens = text.split()
    tokens = set(tokens)
    return tokens

def mkNFC(token):
    for (ch1,ch2) in NFC:
        token.replace(ch1,ch2)
    return token

# turn a txt into clean tokens
def mk_tokens(txt, stw=None, lma=None, ngram=1, how="all", ch_ngram=2, ch_how=None):
    # split into tokens by white space
    tokens = txt.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation+"".join(punc))
    tokens = [w.translate(table) for w in tokens]
    tokens = [mkNFC(token) for token in tokens]
    if stw!=None:
        # filter out stop words
        stop_words = set(read_stopwords('stopword_Urmono.txt'))
        tokens = [w for w in tokens if not w in stop_words]
        # filter out short tokens
        tokens = [word for word in tokens if len(word) > 1]
        tokens = [word for word in tokens if word!='کرنا']
        tokens = [word for word in tokens if word!='پرنا']
        tokens = [word for word in tokens if word!='رہنا']
        tokens = [word for word in tokens if word!='جانا']
        tokens = [word for word in tokens if word!='کَیا']
    if lma!= None:
        #apply lemmatization
        tokens = [lemma(token, dictionary) for token in tokens]
    
    
    #getting the char ngrams
    charngrams=[]
    if ch_how!=None:
        charngrams = mkCNgram(tokens, ch_ngram, ch_how)
    
    wordngrams = mkNgram(tokens, ngram, how)
    tokens = charngrams + wordngrams
    
    #print(tokens)
    return tokens

def mk_dict_from_dataset(txt, stw=None, lma=None, ngram=1, how="all", 
                         ch_ngram=2, ch_how=None):
    c=1
    table = dict()
    for line in txt.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid = cells[0].strip()
        ctext = ' '.join(cells[1:])
        tokens = mk_tokens(ctext, stw, lma, ngram, how, ch_ngram, ch_how)
        table[cid] = tokens
    return table


# load doc and add to vocab
def add_doc_to_vocab(filename, vocab, stw=None, lma=None, 
                     ngram=1, how="all", ch_ngram=2, ch_how=None):
    # load file
    file = load_file(filename)
    # clean doc
    #print("ch_ngram, ch_how:", ch_ngram, ch_how)
    table = mk_dict_from_dataset(file, stw, lma, ngram, how, ch_ngram, ch_how)
    for cid in table:
        tokens = table[cid]
        vocab.update(tokens)

def process_dataset(txt, vocab, train=True, stw=None, lma=None, 
                    ngram=1, how="all", ch_ngram=2, ch_how=None):
    c=1
    table = dict()
    for line in txt.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid = cells[0].strip()
        if (train):
            ctext = ' '.join(cells[1:-1])
            tokens = mk_tokens(ctext, stw, lma, ngram, how, ch_ngram, ch_how)
            clabel = int(cells[2].strip())
        else:
            ctext = ' '.join(cells[1:])
            tokens = mk_tokens(ctext, stw, lma, ngram, how, ch_ngram, ch_how)
            clabel = None #for test
        #print(clabel)
        table[cid] = (tokens, clabel)
        #            {id : (text,1/0)}
    return table

def extract_text_label_from_dict(table):
    txts=list()
    labels=list()
    for item in table:
        (txt, clabel) = table[item]
        txts.append(txt)
        labels.append( int(clabel) )
    return txts, labels

def extract_ids_from_dict(table):
    return list(table.keys())

def mkNgram(tokens, n=1, how="all"):
    if how=="all":
        grams = list()
        for r in range(n):
            r+=1
            grams.extend(["".join( tokens[i:i+r] ) for i in range(len(tokens)-r+1)]) 
        return grams
    else:
        return ["".join( tokens[i:i+n] ) for i in range(len(tokens)-n+1)]


#char ngrams
def mkCNgram(tokens, L, cn_how="all"):
    tokens = "".join( tokens[:] )
    tokens = mkNgram(tokens, L, cn_how)
    return tokens


def mk_test_splits(test_ans_slpit_csv, testcsv, vocab, stw=1, lma=1, 
                   ngram=3, how="all",ch_ngram=2, ch_how=None):
    c=1
    test_x_public = list()
    test_y_public = list()
    test_x_private = list()
    test_y_private = list()
    
    table = process_dataset(testcsv, vocab, False, stw, lma, ngram, how, ch_ngram, ch_how)
    for line in test_ans_slpit_csv.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid   = cells[0].strip()
        label = int( cells[1].strip() )
        split = cells[2].strip()
        if split=='public':
            (tokens, _) = table[cid]
            test_x_public.append( tokens )
            test_y_public.append ( label )
        else:
            (tokens, _) = table[cid]
            test_x_private.append( tokens )
            test_y_private.append ( label )
    return (test_x_public, test_y_public, test_x_private, test_y_private)
        
def mk_test(test_ans_slpit_csv, testcsv, vocab, stw=1, lma=1, 
                   ngram=3, how="all",ch_ngram=2, ch_how=None):
    c=1
    test_x = list()
    test_y = list()
    
    table = process_dataset(testcsv, vocab, False, stw, lma, ngram, how, ch_ngram, ch_how)
    for line in test_ans_slpit_csv.splitlines():
        #print(line)
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid   = cells[0].strip()
        label = cells[1].strip()
        split = cells[2].strip()
        (tokens, _) = table[cid]
        test_x.append( tokens )
        test_y.append ( int(label) )
    
    return (test_x, test_y)

In [36]:
# define vocab
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduAbusive/train.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduAbusive/test.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, 
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            

                            
table = process_dataset(load_file('UrduAbusive/train.csv'), vocab, 
                        train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                        ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print(len(y))
counter = Counter(y)
print(counter)

# load all the test comments from test.csv
testcsv = load_file('UrduAbusive/test.csv')       
test_ans_slpit_csv = load_file('UrduAbusive/test_with_ans_and_split.csv')

(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, 
                           stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                           ch_ngram=v_ch_ngram, ch_how=v_ch_how)
print(len(test_y))
counter = Counter(test_y)
print(counter)

2400
Counter({0: 1213, 1: 1187})
1100
Counter({1: 563, 0: 537})


In [25]:
v_stw=1 
v_lma=1
v_ngram=1
v_how = "all"
v_mode="freq"
v_ch_ngram=2
v_ch_how=None

print("Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1\n")                        
t0 = time.clock()
print()
# define vocab
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduAbusive/train.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduAbusive/test.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, 
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(2))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))
                            
table = process_dataset(load_file('UrduAbusive/train.csv'), vocab, 
                        train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                        ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print("A sample doc and its y:", X_docs[0], y[0])
keys = extract_ids_from_dict(table)
#print(keys[0]) 

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)
X = tokenizer.texts_to_matrix(X_docs, mode=v_mode)

# define vocabulary size (largest integer value)
vector_size = len(tokenizer.word_index) + 1
#print("sentence vector size:", vector_size)
                            

train_x = X
train_y = array(y)
n_words = train_x.shape[1]

# load all the test comments from test.csv
testcsv = load_file('UrduAbusive/test.csv')       
test_ans_slpit_csv = load_file('UrduAbusive/test_with_ans_and_split.csv')

(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, 
                           stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                           ch_ngram=v_ch_ngram, ch_how=v_ch_how)


#print(test_x[])

# encode training data set
Xtest = tokenizer.texts_to_matrix(test_x, mode=v_mode)
#print(Xtest)
                        

#models
models =list()
models.append(svm.SVC(kernel='sigmoid', probability=True))
models.append(svm.SVC(probability=True))
models.append(svm.SVC(kernel='poly',degree=1, probability=True))
models.append(svm.SVC(kernel='poly',degree=2, probability=True))
models.append(svm.SVC(kernel='poly',degree=3, probability=True))


mnames = list()
mnames.append("svm.SVC(kernel='sigmoid', probability=True)")
mnames.append("svm.SVC(probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=1, probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=2, probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=3, probability=True)")

idx=0
for model in models:
    model.fit(train_x,train_y)

    # make probability predictions with the model
    test_pred_labels = model.predict(Xtest)
    
    print(test_pred_labels[0:13])
    print("====")
    print(test_y[0:13])

    predictions_labels=test_pred_labels
    #predictions_labels = list()
    #for v in test_pred_labels:
    #    predictions_labels.append( int(v) )


    f1 = f1_score(test_y, predictions_labels)
    strf1 = str(f1*100)
    
   

    t1 = time.clock() - t0
    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how)+","+str(v_mode)+","+mnames[idx]+","+str(len(vocab))+","+str(vector_size)+","+str(t1)+","+strf1
    idx+=1
    print(msg)
    print("=========================================================================")

Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1


Most common: [('کرنا', 779), ('حرام', 352)]
Vocabulary Size: 5695
A sample doc and its y: ['قوم', 'پرانا', 'خاندانی', 'نمک', 'حرام'] 1


In [23]:
#a simple MLP model to predict the sentiment of encoded reviews.
#f1 around ~78 max
def mkMLPExp(opt_stopwords, opt_lemma, opt_ngram, opt_how_ngram, opt_mode, ch_ngram, ch_how):
    print("Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1\n")                        
    for v_stw in opt_stopwords:
        for v_lma in opt_lemma:
            for v_ngram in opt_ngram:
                for v_how_ngram in opt_how_ngram:
                    
                    for v_mode in opt_mode:
                        model="MLP"
                        t0 = time.clock()
                        print()
                        print()
                        # define vocab
                        vocab = Counter()
                        # add all text to vocab
                        add_doc_to_vocab('UrduAbusive/train.csv', vocab, stw=v_stw, lma=v_lma, 
                                             ngram=v_ngram, how=v_how_ngram, 
                                                 ch_ngram=ch_ngram, ch_how=ch_how)
                        add_doc_to_vocab('UrduAbusive/test.csv', vocab, stw=v_stw, lma=v_lma, 
                                             ngram=v_ngram, how=v_how_ngram, 
                                                 ch_ngram=ch_ngram, ch_how=ch_how)
                            
                        print("Most common:", vocab.most_common(2))
                        vocab = set(vocab)
                        print("Vocabulary Size:", len(vocab))
                            
                        table = process_dataset(load_file('UrduAbusive/train.csv'), vocab, 
                                                    train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how_ngram,
                                                     ch_ngram=ch_ngram, ch_how=ch_how)

                        X_docs, y = extract_text_label_from_dict(table)
                        print("A sample doc and its y:", X_docs[0], y[0])
                        keys = extract_ids_from_dict(table)
                        
                        # create the tokenizer
                        tokenizer = Tokenizer()
                        # fit the tokenizer on the documents
                        tokenizer.fit_on_texts(X_docs)
                        X = tokenizer.texts_to_matrix(X_docs, mode=v_mode)

                        # define vocabulary size (largest integer value)
                        vector_size = len(tokenizer.word_index) + 1
                        #print("sentence vector size:", vector_size)
                            

                        train_x = X
                        train_y = array(y)

                        
                        n_words = train_x.shape[1]

                        # define network
                        model = Sequential()
                        model.add(Dense(50, input_shape=(n_words,), activation='relu'))
                        model.add(Dense(25, input_shape=(n_words,), activation='relu'))
                        model.add(Dense(1, activation='sigmoid'))
                        # compile network
                        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
                        # fit network
                        model.fit(train_x, train_y, epochs=50, verbose=0)
                        
                        
                        # load all the test comments from test.csv and split in public and private sets
                        testcsv = load_file('UrduAbusive/test.csv')       
                        test_ans_slpit_csv = load_file('UrduAbusive/test_with_ans_and_split.csv')

                        (test_x, test_y) = mk_test(
                            test_ans_slpit_csv, testcsv, vocab, stw=v_stw, lma=v_lma, ngram=v_ngram, 
                                how=v_how_ngram, ch_ngram=ch_ngram, ch_how=ch_how)

                          # encode training data set
                        Xtest = tokenizer.texts_to_matrix(test_x, mode=v_mode)
                        test_y = array(test_y)
                        
                        # make probability predictions with the model
                        predictions_scores = model.predict(Xtest)
                        predictions_labels = model.predict_classes(Xtest)

                        
                        predictions_labels = predictions_labels[:, 0]
                        f1 = f1_score(test_y, predictions_labels)

                        strf1 = str(f1*100)
                            
                        t1 = time.clock() - t0
                        msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how_ngram)+","+str(v_mode)+",Simple MLP,"+str(len(vocab))+","+str(vector_size)+","+str(t1)+","+strf1
                            
                        print(msg)
                        print("=========================================================================")

In [27]:
opt_stopwords =[1] #[None, 1]
opt_lemma =[1] #[None, 1]
opt_ngram = [2] #[1,2,3]
opt_how_ngram = ["all"] #"exact",
opt_mode = ["freq"] #, "count", "freq", "tfidf"]

for i in range(10):
    mkMLPExp(opt_stopwords, opt_lemma, opt_ngram, opt_how_ngram, opt_mode, 2, "exact")
    

Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1



Most common: [('نا', 4913), ('ان', 3101)]
Vocabulary Size: 26730
A sample doc and its y: ['قو', 'وم', 'مپ', 'پر', 'را', 'ان', 'نا', 'اخ', 'خا', 'ان', 'ند', 'دا', 'ان', 'نی', 'ین', 'نم', 'مک', 'کح', 'حر', 'را', 'ام', 'قوم', 'پرانا', 'خاندانی', 'نمک', 'حرام', 'قومپرانا', 'پراناخاندانی', 'خاندانینمک', 'نمکحرام'] 1
1,1,2,all,freq,Simple MLP,26730,20485,82.80178280000109,76.13741875580317
Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1



Most common: [('نا', 4913), ('ان', 3101)]
Vocabulary Size: 26730
A sample doc and its y: ['قو', 'وم', 'مپ', 'پر', 'را', 'ان', 'نا', 'اخ', 'خا', 'ان', 'ند', 'دا', 'ان', 'نی', 'ین', 'نم', 'مک', 'کح', 'حر', 'را', 'ام', 'قوم', 'پرانا', 'خاندانی', 'نمک', 'حرام', 'قومپرانا', 'پراناخاندانی', 'خاندانینمک', 'نمکحرام'] 1
1,1,2,all,freq,Simple MLP,26730,20485,78.44287610000174,74.10281280310377
Stopwords,Lemmatization,NGram,How NGram,

# Word2Vec Results with and without SMOTE

In [20]:
import numpy as np 
import pandas as pd
import gensim

v_stw = 1 #1 # None #
v_lma = 1  
v_ngram = 2
v_how = "all"
v_ch_ngram = 2
v_ch_how = None

url='Urdu-word2vec-implementations/samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

print("Stopwords,Lemmatization,NGram,How NGram,Model,Vocab size,Vector size,Time elapsed,F1\n")                        

# define vocab  
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduAbusive/train.csv', vocab, stw=v_stw, lma=v_lma, 
                 ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduAbusive/test.csv', vocab, stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(5))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))

table = process_dataset(load_file('UrduAbusive/train.csv'), vocab, train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs_train, y = extract_text_label_from_dict(table)
ytrain = y

print("A sample doc and its y:", X_docs_train[0], y[0])
print("len:", len(X_docs_train[0]))

zero=list()
one=list()
for ty in ytrain:
    if ty==0:
        zero.append(0)
    elif ty==1:
        one.append(1)
    else:
        print(ty)
        
print ( "0->", len(zero))
print ( "1->", len(one)) 
print(len(X_docs_train)-len(zero))

Stopwords,Lemmatization,NGram,How NGram,Model,Vocab size,Vector size,Time elapsed,F1

Most common: [('کرنا', 779), ('حرام', 352), ('کتا', 342), ('بکواس', 324), ('مادرچود', 292)]
Vocabulary Size: 24949
A sample doc and its y: ['قوم', 'پرانا', 'خاندانی', 'نمک', 'حرام', 'قومپرانا', 'پراناخاندانی', 'خاندانینمک', 'نمکحرام'] 1
len: 9
0-> 1213
1-> 1187
1187


In [23]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Abusive'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Abusive', axis = 1)
train_y = docs_vectors['Abusive']

print("total instances:", len(train_x))
##rint(len(train_y))

#print(train_x)
#print(train_y)

# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
#oversample = SMOTE()
#X, y = oversample.fit_resample(train_x, train_y)
X=train_x
y=train_y

# summarize the new class distribution
counter = Counter(y)
print(counter)


# load all the test comments from test.csv 
testcsv = load_file('UrduAbusive/test.csv')       
test_ans_slpit_csv = load_file('UrduAbusive/test_with_ans_and_split.csv')
(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, stw=v_stw, lma=v_lma, ngram=v_ngram, 
                              how=v_how, ch_ngram=v_ch_ngram, ch_how=v_ch_how)
            
X_docs_test = test_x
test_y = array(test_y)

tdocs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_test:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    tdoc_vector = temp.mean() 
    tdocs_vectors = tdocs_vectors.append(tdoc_vector, ignore_index = True)
    
print(tdocs_vectors.shape)

pd.isnull(tdocs_vectors).sum().sum()

tdocs_vectors = tdocs_vectors.fillna(0)

#tdocs_vectors
Xtest = tdocs_vectors


models =list()
models.append(svm.SVC(kernel='poly',degree=1, probability=True))
models.append(svm.SVC(kernel='poly',degree=2, probability=True))
models.append(svm.SVC(kernel='poly',degree=3, probability=True))
models.append(svm.SVC(probability=True))
models.append(svm.SVC(kernel='sigmoid', probability=True))

names = list()
names.append("svm.SVC(kernel='poly',degree=1, probability=True)")
names.append("svm.SVC(kernel='poly',degree=2, probability=True)")
names.append("svm.SVC(kernel='poly',degree=3, probability=True)")
names.append("svm.SVC(probability=True)")
names.append("svm.SVC(kernel='sigmoid', probability=True)")

idx=0
for model in models:
    model.fit(X,y)

    # make probability predictions with the model
    test_pred_labels = model.predict(Xtest)
    #print(test_pred_labels[0:3])

    predictions_labels = list()
    for v in test_pred_labels:
        predictions_labels.append( int(v) )


    f1 = f1_score(test_y, predictions_labels)
    strf1 = str(f1*100)
    vector_size=300
    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how)+","+",Word2Vec,"+names[idx]+str(len(vocab))+","+str(vector_size)+","+strf1
    idx+=1
    print(msg)
    print("=========================================================================")


(2400, 300)
total instances: 2400
Counter({0: 1213, 1: 1187})
(1100, 300)
1,1,2,all,,Word2Vec,svm.SVC(kernel='poly',degree=1, probability=True)24949,300,78.05325987144168
1,1,2,all,,Word2Vec,svm.SVC(kernel='poly',degree=2, probability=True)24949,300,79.12087912087912
1,1,2,all,,Word2Vec,svm.SVC(kernel='poly',degree=3, probability=True)24949,300,78.55153203342618
1,1,2,all,,Word2Vec,svm.SVC(probability=True)24949,300,79.16287534121929
1,1,2,all,,Word2Vec,svm.SVC(kernel='sigmoid', probability=True)24949,300,75.78558225508317


pandas.core.frame.DataFrame

# Deep Convolutional Neural Network for Sentiment Analysis

In [ ]:
#a Deep CNN model to predict the sentiment of encoded reviews.
#f1 around 
def mkCNNExp(opt_stopwords, opt_lemma, opt_ngram, opt_how_ngram, opt_mode, ch_ngram, ch_how):
    print("Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1\n")                        
    for v_stw in opt_stopwords:
        for v_lma in opt_lemma:
            for v_ngram in opt_ngram:
                for v_how_ngram in opt_how_ngram:
                    
                    for v_mode in opt_mode:
                        t0 = time.clock()
                        print()
                        print()
                        # define vocab
                        vocab = Counter()
                        # add all text to vocab
                        add_doc_to_vocab('UrduAbusive/train.csv', vocab, stw=v_stw, lma=v_lma, 
                                             ngram=v_ngram, how=v_how_ngram, 
                                                 ch_ngram=ch_ngram, ch_how=ch_how)
                        add_doc_to_vocab('UrduAbusive/test.csv', vocab, stw=v_stw, lma=v_lma, 
                                             ngram=v_ngram, how=v_how_ngram, 
                                                 ch_ngram=ch_ngram, ch_how=ch_how)
                            
                        print("Most common:", vocab.most_common(2))
                        vocab = set(vocab)
                        print("Vocabulary Size:", len(vocab))
                            
                        table = process_dataset(load_file('UrduAbusive/train.csv'), vocab, 
                                                    train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how_ngram,
                                                     ch_ngram=ch_ngram, ch_how=ch_how)

                        X_docs, y = extract_text_label_from_dict(table)
                        print("A sample doc and its y:", X_docs[0], y[0])
                        
                        
                        # create the tokenizer
                        tokenizer = Tokenizer()
                        # fit the tokenizer on the documents
                        tokenizer.fit_on_texts(X_docs)
                        # encode training data set
                        encoded_docs = tokenizer.texts_to_sequences(X_docs)
                        # pad sequences
                        max_length = max([len(s) for s in X_docs])
                        #print(max_length)

                            
                        Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
                        ytrain = array(y)

                        vocab_size = len(tokenizer.word_index) + 1
                        vector_size = vocab_size
                        train_x = Xtrain
                        train_y = ytrain
                        
                        n_words = train_x.shape[1]

                        # define network
                        model = Sequential()
                        model.add(  Embedding(vocab_size, 100, input_length=max_length)  )
                        model.add(  Conv1D(filters=32, kernel_size=8, activation='relu') )
                        model.add(  MaxPooling1D(pool_size=2) )
                        model.add(  Flatten())
                        model.add(  Dense(10, activation='relu'))
                        model.add(  Dense(1, activation='sigmoid'))
                        print(model.summary())

                        # compile network
                        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
                        # fit network
                        model.fit(train_x, train_y, epochs=50, verbose=0)
                        
                        
                        
                        # load all the test comments from test.csv and split in public and private sets
                        testcsv = load_file('UrduAbusive/test.csv')       
                        test_ans_slpit_csv = load_file('UrduAbusive/test_with_ans_and_split.csv')

                        (test_x, test_y) = mk_test(
                            test_ans_slpit_csv, testcsv, vocab, stw=v_stw, lma=v_lma, ngram=v_ngram, 
                                how=v_how_ngram, ch_ngram=ch_ngram, ch_how=ch_how)

                        
                        
                        X_docs_test = test_x
                        # sequence encode
                        encoded_docs_test = tokenizer.texts_to_sequences(X_docs_test)
                        # pad sequences
                        Xtest = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
                        test_y = array(test_y)
                        
                        
                        # make probability predictions with the model
                        predictions_scores = model.predict(Xtest)
                        predictions_labels = model.predict_classes(Xtest)

                        
                        predictions_labels = predictions_labels[:, 0]
                        f1 = f1_score(test_y, predictions_labels)

                        strf1 = str(f1*100)
                            
                        t1 = time.clock() - t0
                        msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","
                        msg+= str(v_how_ngram)+","+str(v_mode)+",Deep CNN,"
                        msg+= str(len(vocab))+","+str(vector_size)+","
                        msg+= str(t1)+","+strf1
                            
                        print(msg)
                        print("=========================================================================")
                        


In [ ]:
opt_stopwords =[1] #[None, 1]
opt_lemma =[1] #[None, 1]
opt_ngram = [3] #[1,2,3]
opt_how_ngram = ["all"] #"exact",
opt_mode = ["binary", "freq"] #, "count", "freq", "tfidf"]

mkCNNExp(opt_stopwords, opt_lemma, opt_ngram, opt_how_ngram, opt_mode, 2, None)

# Develop an n-gram CNN Model for Sentiment Analysis
### "CNN Word2Vec NGram"

In [21]:
#N gram CNN model

from numpy import array
import numpy
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(lines)
    return tokenizer

# calculate the maximum document length
def max_length(lines):
    return max([len(s) for s in lines])


# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length,  padding='post')
    return padded

# define the model
def define_model(length, vocab_size):
    
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(32, 1, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D()(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(32, 2, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D()(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(32, 3, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D()(drop3)
    flat3 = Flatten()(pool3)
    # channel 4
    inputs4 = Input(shape=(length,))
    embedding4 = Embedding(vocab_size, 100)(inputs4)
    conv4 = Conv1D(32, 4, activation='relu')(embedding4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling1D()(drop4)
    flat4 = Flatten()(pool4)
    # merge
    merged = concatenate([flat1, flat2, flat3, flat4])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    #model.summary()
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 2
v_ch_how = None

t0 = time.clock()
print()
print()

# define vocab  
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduAbusive/train.csv', vocab, stw=v_stw, lma=v_lma, 
                 ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduAbusive/test.csv', vocab, stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(2))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))



    
table = process_dataset(load_file('UrduAbusive/train.csv'), vocab, train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print("A sample doc and its y:", X_docs[0], y[0])
y = array(y)

## create tokenizer
tokenizer = create_tokenizer(X_docs)
# calculate max document length
length = max_length(X_docs)
## load all training reviews
X = encode_text(tokenizer, X_docs, length)

# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1

train_x = X

# calculate max document length
length = max_length(train_x)
print('Max document length: %d' % length)
    
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = train_x

# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX,trainX], y, epochs=10, batch_size=7)

# load all the test comments from test.csv and split in public and private sets
testcsv = load_file('UrduAbusive/test.csv')       
test_ans_slpit_csv = load_file('UrduAbusive/test_with_ans_and_split.csv')

(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, ch_ngram=v_ch_ngram, ch_how=v_ch_how)

                        
Xtest = encode_text(tokenizer, test_x, length)
test_y = array(test_y)

# make probability predictions with the model
predictions_scores = model.predict([Xtest,Xtest,Xtest,Xtest])


predictions_labels = list()
for score in predictions_scores:
    if score>=0.5:
        predictions_labels.append( int(1) )
    else:
        predictions_labels.append( int(0) )
        
f1 = f1_score(test_y, predictions_labels)

strf1 = str(f1*100)

t1 = time.clock() - t0

msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","
msg+= str(v_how)+","+"NA"+",\"Very Deep CNN\","
msg+= str(len(vocab))+","+str(vocab_size)+","
msg+= str(t1)+","+strf1
                            
print(msg)
print("=========================================================================")



Most common: [('کرنا', 779), ('حرام', 352)]
Vocabulary Size: 5695
A sample doc and its y: ['قوم', 'پرانا', 'خاندانی', 'نمک', 'حرام'] 1
Max document length: 27
Vocabulary size: 4657


C:\Users\mhumayoun\Anaconda3\envs\mle\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
2400/2400 [==============================] - 13s 5ms/step - loss: 0.5344 - accuracy: 0.7204
Epoch 2/10
2400/2400 [==============================] - 13s 5ms/step - loss: 0.2780 - accuracy: 0.8950
Epoch 3/10
2400/2400 [==============================] - 13s 5ms/step - loss: 0.1725 - accuracy: 0.9362
Epoch 4/10
2400/2400 [==============================] - 13s 5ms/step - loss: 0.1272 - accuracy: 0.9546
Epoch 5/10
2400/2400 [==============================] - 12s 5ms/step - loss: 0.1032 - accuracy: 0.9533
Epoch 6/10
2400/2400 [==============================] - 12s 5ms/step - loss: 0.0774 - accuracy: 0.9558
Epoch 7/10
2400/2400 [==============================] - 12s 5ms/step - loss: 0.0736 - accuracy: 0.9592
Epoch 8/10
2400/2400 [==============================] - 12s 5ms/step - loss: 0.0623 - accuracy: 0.9583
Epoch 9/10
2400/2400 [==============================] - 13s 5ms/step - loss: 0.0557 - accuracy: 0.9646
Epoch 10/10
2400/2400 [==============================] - 14s 6ms/step - l

## Second Submission: Bag of words SVM segmoid
#### UrduAbusive_results_1.csv F1:0.80548	ROC_AUCC: 0.82061

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

punc=['…','،','۔','،','۔','؟' , '.', ',']

# load the vocabulary
vocab_filename = 'vocabUrdu.txt'
#vocab_filename = 'vocabUrduAbusive_no_stw_removal_allwords.txt' 
#UrduAbusive_results_1_1.csv
#F1 0.77871 AUC: 0.79999

vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(len(vocab))

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduAbusive/train.csv'), vocab)

X_docs = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)

X = tokenizer.texts_to_matrix(X_docs, mode='freq')
y = array(extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1))

print(len(X[0]))
      

train_x = X
train_y = y

model = svm.SVC(kernel='sigmoid', probability=True).fit(train_x, train_y)


#f1 = f1_score(test_y, test_pred)
#res_SVM_segmoid_f1.append(f1*100)

# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduAbusive/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

# encode training data set
Xtest = tokenizer.texts_to_matrix(X_docs_test, mode='freq')

# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))
       

#from sklearn.model_selection import cross_val_score
#cross_val_score(model, X, y, cv=5, scoring='f1')

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+"{:.5f}".format(test_pred_scores[i])+'\n'

#file = open('UrduAbusive/results/UrduAbusive_results_1_1.csv', 'w', encoding='utf8')
#file.write(results)
#file.close()

In [ ]:
# Develop Super Learner Ensembles in Python
# Classification for abusive language problem
# Classification
# example of a super learner using the mlens library
# F1 0.77528 AUCC 0.5

# it did not perform on Absuive dataset
# may be add better classifers
#https://machinelearningmastery.com/super-learner-ensemble-in-python/
#How to make it better?
# use only the best performing algos

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from mlens.ensemble import SuperLearner

# create a list of base-models
def get_models():
    models = list()
    models.append(LogisticRegression(solver='liblinear'))
    models.append(DecisionTreeClassifier())
    models.append(SVC(gamma='scale', probability=True))
    #models.append(svm.SVC(kernel='sigmoid'))
    #models.append(svm.SVC(), probability=True)
    models.append(GaussianNB())
    models.append(KNeighborsClassifier())
    models.append(AdaBoostClassifier())
    models.append(BaggingClassifier(n_estimators=10))
    models.append(RandomForestClassifier(n_estimators=10))
    models.append(ExtraTreesClassifier(n_estimators=10))
    return models
 
# create the super learner
def get_super_learner(X):
    ensemble = SuperLearner(scorer=accuracy_score, folds=10, shuffle=True, sample_size=len(X))
    # add base models
    models = get_models()
    ensemble.add(models)
    # add the meta model
    ensemble.add_meta(LogisticRegression(solver='lbfgs'))
    return ensemble


# load the vocabulary
vocab_filename = 'vocabUrdu.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduAbusive/train.csv'), vocab)

X_docs = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)

X = tokenizer.texts_to_matrix(X_docs, mode='freq')
y = array(extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1))

# split
X, X_val, y, y_val = train_test_split(X, y, test_size=0.50)
print('Train', X.shape, y.shape, 'Test', X_val.shape, y_val.shape)
# create the super learner
ensemble = get_super_learner(X)
# fit the super learner
ensemble.fit(X, y)
# summarize base learners
print(ensemble.data)
# make predictions on hold out set
yhat = ensemble.predict(X_val)
print(yhat[0:10])
print('F1 Super Learner: %.3f' % (f1_score(y_val, yhat) * 100))

#accuracy Super Learner: 83.000
#F1 Super Learner: 83.406
# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduAbusive/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

# encode training data set
Xtest = tokenizer.texts_to_matrix(X_docs_test, mode='freq')

# make probability predictions with the model
test_pred_labels = ensemble.predict(Xtest)
#test_pred_scr = ensemble.predict_probb(Xtest)

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_labels)):
    results += str(ids[i])+","+str(int(test_pred_labels[i]))+","+"0.0"+'\n'

file = open('UrduAbusive/results/UrduAbusive_results_4.csv', 'w', encoding='utf8')
file.write(results)
file.close()

In [ ]:
# stacking ensemble

In [ ]:
#voting based ensenble

In [ ]:
# load the vocabulary
vocab_filename = 'vocabUrdu.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduAbusive/train.csv'), vocab)

X_docs = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)

X_docs = trim_docs(X_docs, 4)

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)

X = tokenizer.texts_to_matrix(X_docs, mode='freq')
y = array(extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1))

res_SVM_segmoid_f1=[]

train_x = X
train_y = y

model = svm.SVC(kernel='sigmoid', probability=True).fit(train_x, train_y)


# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduAbusive/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

X_docs_test = trim_docs(X_docs_test, 4)


# encode training data set
Xtest = tokenizer.texts_to_matrix(X_docs_test, mode='freq')

# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))
       

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+"{:.5f}".format(test_pred_scores[i])+'\n'

file = open('UrduAbusive/results/UrduAbusive_results_3.csv', 'w', encoding='utf8')
file.write(results)
file.close()


In [ ]:
# Result got worse with SMOTE

# load the vocabulary
vocab_filename = 'vocabUrdu.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduAbusive/train.csv'), vocab)

X_docs = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)


# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)

X = tokenizer.texts_to_matrix(X_docs, mode='freq')
y = array(extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1))


import imblearn
from imblearn.over_sampling import SMOTE
# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


res_SVM_segmoid_f1=[]

train_x = X
train_y = y

model = svm.SVC(kernel='sigmoid', probability=True).fit(train_x, train_y)


#f1 = f1_score(test_y, test_pred)
#res_SVM_segmoid_f1.append(f1*100)

# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduAbusive/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

# encode training data set
Xtest = tokenizer.texts_to_matrix(X_docs_test, mode='freq')

# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))
       
ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+"{:.5f}".format(test_pred_scores[i])+'\n'

file = open('UrduAbusive/results/UrduAbusive_results_2.csv', 'w', encoding='utf8')
file.write(results)
file.close()

In [ ]:
# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduAbusive/train.csv'), vocab)

X_docs = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)
X_docs_trim = trim_docs(X_docs, 3)
X_docs_trim[0]

In [ ]:
# load the vocabulary
vocab_filename = 'vocabUrdu.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(len(vocab))